# Joints 2 SMPL 映射关系

需要通过手动填写 keys_mapping_new.csv 文件确定导出 Joints 列表与 SMPL 骨骼名的映射关系

如下所示：

```csv
order,smpl_name,fbx_name
0,pelvis,m_avg_Pelvis
1,left leg root,m_avg_L_Hip
2,right leg root,m_avg_R_Hip
```

In [18]:
import numpy as np
import pandas as pd
import json

pelvis_joint_name = "m_avg_Pelvis"

# 获取映射表
mapping_table = pd.read_csv("keys_mapping.csv")
mapping_table.set_index('fbx_name', inplace=True)

# mapping_table

In [19]:
import os

def get_changed_extention_abs_path(abs_path,ext):
    return os.path.splitext(abs_path)[0]+"."+ext.replace(".","")

In [28]:
from scipy.spatial.transform import Rotation

def json2amass_npz(json_path, mapping_table):

    with open(json_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
        frame_length = data["frame_length"]

        # 通用
        gender = 'male'                         # 性别
        mocap_framerate = 30                    # 帧率
        betas = np.zeros(16)                    # 形状参数，默认为0
        dmpls = np.zeros((frame_length, 8))     # 软组织系数，默认为0

        # 更新
        trans = np.zeros((frame_length, 3))     # 全局平移
        poses = np.zeros((frame_length, 156))   # 姿势参数

        # 处理每帧数据
        for frame_data in data["frame_sequence"]:
            # 第几帧索引
            frame_index = frame_data["frame_number"]

            pelvis_position = np.zeros(3)

            joint_positions = {}
            joint_rotations = {}

            # 处理关节点数据
            for joint_data in frame_data["joint_position"]:

                joint_name = joint_data["joint_name"]
                # 尝试获取当前关节序号

                if joint_name not in mapping_table.index:
                    continue


                order = mapping_table.loc[joint_name]["order"]
                joint_positions[order] = np.array([joint_data["position_x"],
                                                -joint_data["position_z"],
                                                joint_data["position_y"]])

                if joint_name == pelvis_joint_name:
                    pelvis_position = joint_positions[order]

                # 获取欧拉角
                euler_angles = np.array([joint_data["rotation_x"],
                                        joint_data["rotation_y"],
                                        joint_data["rotation_z"]])

                euler_angles = (euler_angles + 180) % 360 - 180
                # 将欧拉角转换为轴角表示法
                joint_rotations[order] = Rotation.from_euler('xyz', euler_angles, degrees=True).as_rotvec()
                        

                
                
            for order in joint_rotations.keys():
                poses[frame_index, order * 3:(order + 1) * 3] = joint_rotations[order]

            # 计算m_avg_Pelvis和m_avg_root累计的位置变化
            
            trans[frame_index, :] = pelvis_position * 0.01

    # 将数据保存为AMASS格式（NPZ文件）
    np.savez(get_changed_extention_abs_path(json_path, ".npz"), trans=trans, gender=gender, mocap_framerate=mocap_framerate, betas=betas, dmpls=dmpls, poses=poses)

In [32]:
import os
import glob
from tqdm import tqdm

print(mapping_table.columns)


# 指定目录路径
dir_path = r'F:\AI Repository\Motion Diffusion\SoulsAnim\ER'

# 获取目录中所有的 JSON 文件
json_files = glob.glob(os.path.join(dir_path, '*.json'))

# 使用 tqdm 进度条包装 json_files 列表
for i, file_path in enumerate(tqdm(json_files, desc="Processing files", unit="file")):
    # print(os.path.basename(file_path))
    json2amass_npz(file_path, mapping_table)


Index(['order', 'smpl_name'], dtype='object')


Processing files: 100%|██████████| 7644/7644 [29:35<00:00,  4.31file/s] 


In [ ]:
import os
import shutil

# 源目录和目标目录
source_dir = 'F:\AI Repository\Motion Diffusion\SoulsAnim'
dest_dir = 'F:\AI Repository\Motion Diffusion\SoulsAmass'

# 遍历源目录下的所有文件和子目录
for root, dirs, files in os.walk(source_dir):
    # 对于每个文件
    for file in files:
        # 如果文件是以'.npz'结尾的
        if file.endswith('.npz'):
            # 计算文件的相对路径
            rel_path = os.path.relpath(os.path.join(root, file), source_dir)
            # 构造目标路径
            dest_path = os.path.join(dest_dir, rel_path)
            # 如果目标路径不存在，则创建目录
            if not os.path.exists(os.path.dirname(dest_path)):
                os.makedirs(os.path.dirname(dest_path))
            # 复制文件
            shutil.copyfile(os.path.join(root, file), dest_path)
